In [1]:
import requests
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import joblib
import time
from datetime import datetime, timedelta

In [2]:
'''
    Import the pre-trained ML model
'''

scaler = joblib.load('scaler.joblib')
kmeans = joblib.load('kmeans_model.joblib')

In [3]:
'''
    The address that we want to score
'''

addresses = ['arsc4jbDnzaqcCLByyGo7fg7S2SmcFsWUzQuDtLZh2y']

In [4]:
accountTxDatas = []

In [5]:
'''
    fetch the address onchain tramsfer txns data using solana.fm api
'''

for i in addresses:
    
    time.sleep(1)
    
    url = f"https://api.solana.fm/v0/accounts/{i}/transfers"
    response = requests.get(url)

    parssedTxResult = []
    if response.status_code == 200:
        data = response.json()
        
        results = data.get('results')

        for tx in results:
            data = tx.get('data',[])
            for i in data:
                # if (i.get('action')).startswith('mintToCollection') :
                    parssedTxResult.append(
                        {
                            'status' : i.get('status'),
                            'action' : i.get('action'),
                            'amount' : i.get('amount'),
                            'counterparty' : i.get('destination'),
                            'timestamp' : datetime.fromtimestamp(i.get('timestamp')),
                        }
                    )

        accountTxDatas.append(parssedTxResult)
        
    else:
        print(f"Request failed with status code {response.status_code}")

In [6]:
def calculate_total_transactions(transactions):
    return len(transactions)

def calculate_daily_transaction_count(transactions):
    daily_count = {}
    
    for tx in transactions:
        date = tx['timestamp'].strftime('%Y-%m-%d')
        if date not in daily_count:
            daily_count[date] = 1
        daily_count[date] += 1
    return daily_count

def calculate_transaction_intervals(transactions):
    intervals = []
    timestamps = [tx['timestamp'] for tx in transactions]
    timestamps.sort()
    for i in range(1, len(timestamps)):
        interval = (timestamps[i] - timestamps[i-1]).total_seconds() / 3600.0
        intervals.append(interval)
    return intervals

def calculate_total_transaction_volume(transactions):
    return sum(tx['amount'] for tx in transactions)

def calculate_mean_transaction_amount(transactions):
    return np.mean([tx['amount'] for tx in transactions])

def calculate_variance_transaction_amount(transactions):
    return np.var([tx['amount'] for tx in transactions])

def calculate_unique_counterparties(transactions):
    counterparties = set(tx['counterparty'] for tx in transactions)
    return len(counterparties)

def detect_high_frequency_bursts(transactions, threshold=1):
    bursts = 0
    intervals = calculate_transaction_intervals(transactions)
    for interval in intervals:
        if interval < threshold:
            bursts += 1
    return bursts

def calculate_success_rate(transactions):
    total_transactions = len(transactions)
    successful_transactions = sum(1 for tx in transactions if tx['status']== 'Successful')
    return successful_transactions / total_transactions if total_transactions > 0 else 0

def calculate_time_of_day_activity(transactions):
    hours = [tx['timestamp'].hour for tx in transactions]
    return np.mean(hours) if hours else 0

def calculate_transfer_counts(transactions):
    transfer_counts = [
        "transfer",
        "transferWithSeed",
        "transferChecked",
        "transferCheckedWithFee"
    ]

    transfer = 0
    for tx in transactions:
        tx_type = tx['action']
        if tx_type in transfer_counts:
            transfer += 1
    
    return transfer/len(transactions)

def calculate_mint_and_sig_counts(transactions):
    account_creation_counts = [
        "mintTo",
        "initializeMint",
        "initializeMint2",
        "initializeMultisig",
        "initializeMultisig2"
    ]

    count = 0
    for tx in transactions:
        tx_type = tx['action']
        if tx_type in account_creation_counts:
            count += 1
    
    return count/len(transactions)

In [7]:
features = []
for transactions in accountTxDatas:
    total_transactions = calculate_total_transactions(transactions)
    daily_transaction_count = calculate_daily_transaction_count(transactions)
    transaction_intervals = calculate_transaction_intervals(transactions)
    total_transaction_volume = calculate_total_transaction_volume(transactions)
    mean_transaction_amount = calculate_mean_transaction_amount(transactions)
    variance_transaction_amount = calculate_variance_transaction_amount(transactions)
    unique_counterparties = calculate_unique_counterparties(transactions)
    high_frequency_bursts = detect_high_frequency_bursts(transactions)
    success_rate = calculate_success_rate(transactions)
    time_of_day_activity = calculate_time_of_day_activity(transactions)

    transfer_counts = calculate_transfer_counts(transactions)
    mint_and_sig_counts = calculate_mint_and_sig_counts(transactions)
    
    feature_vector =[
        total_transactions,
        len(daily_transaction_count), 
        np.mean(transaction_intervals) if transaction_intervals else 0,
        total_transaction_volume,
        mean_transaction_amount,
        variance_transaction_amount,
        unique_counterparties,
        high_frequency_bursts,
        success_rate,
        time_of_day_activity,
        transfer_counts,
        mint_and_sig_counts
        ]
    
    features.append(feature_vector)

In [8]:
features

[[372,
  1,
  0.00013776579814315663,
  793642920364830553,
  2133448710658146.8,
  1.871609965358034e+32,
  16,
  371,
  1.0,
  12.0,
  0.6155913978494624,
  0.0]]

In [9]:

feature_names = [
    'Total_Transactions',
    'Unique_Days',
    'Mean_Transaction_Interval',
    'Total_Transaction_Volume',
    'Mean_Transaction_Amount',
    'Variance_Transaction_Amount',
    'Unique_Counterparties',
    'High_Frequency_Bursts',
    'Success_Rate',
    'Time_Of_Activity',
    'Transfer_Counts',
    'Mint_And_Sig_Counts'
]

In [10]:
features_df = pd.DataFrame(features, columns=feature_names)

scaled_features = scaler.transform(features_df)

# Predict the cluster
predicted_cluster = kmeans.predict(scaled_features)

In [11]:
predicted_cluster

array([1], dtype=int32)